<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mplfinance

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

mypf = pd.merge(mypfs, mypps, on="Symbol")

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
#mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
mypf

,Symbol,AvgCost,Shares,InPortfolio,Target,Breakout,MedianPE,NSECap,Criteria,Strategy,TFrame,LatestQtr,StarStock,BizFinTech,Conviction,Bucket,Category,Stock
0,3MINDIA,NaN,NaN,NaN,NaN,NaN,72.7,0.0,V40N,ATH,D,1,1,1.0,M,NaN,MISC,3MINDIA.NS
1,5PAISA,415.15,148.0,DM,810.0,NaN,65.0,0.0,V40N,BTT,W,1,1,1.0,M,NaN,FINANCE,5PAISA.NS
2,AARTIIND,487.04,167.0,SV,680.0,516.0,31.2,0.0,VR,NTT,W,1,0,0.0,L,NaN,CHEMICALS,AARTIIND.NS
3,AAVAS,1453.92,37.0,DM,1915.0,1666.0,57.7,0.0,VR,NTT,W,1,1,1.0,L,NaN,FINANCE,AAVAS.NS
4,AAVAS,1663.45,36.0,SV,1915.0,1666.0,57.7,0.0,VR,NTT,W,1,1,1.0,L,NaN,FINANCE,AAVAS.NS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,WIPRO,389.70,271.0,DM,533.0,NaN,20.9,1.0,VR,NTT,W,1,1,1.0,L,NaN,IT,WIPRO.NS
250,WIPRO,421.41,150.0,SV,533.0,NaN,20.9,1.0,VR,NTT,W,1,1,1.0,L,NaN,IT,WIPRO.NS
251,WSTCSTPAPR,NaN,NaN,NaN,NaN,NaN,NaN,0.0,VR,ATH,W,0,0,0.0,L,NaN,PAPER,WSTCSTPAPR.NS
252,YESBANK,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1,0,0.0,L,NaN,NaN,YESBANK.NS


In [5]:
import pandas as pd
import mplfinance as mpf
import yfinance as yfin
#import talib
yfin.pdr_override()

def get_mtm_data(symbol, tf):
  if tf == 'D':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1d', 20, 50, 200, '2y', '2023-01-01','Daily'
  elif tf == 'W':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1wk', 5, 10, 30, '6y', '2019-01-01','Weekly'
  else:
    intvl, sma, lma, xlma, prd, start_date, tfc = '1mo', 2, 5, 10, '10y', '2013-01-01','Monthly'

  # Sample data with OHLCV (Open, High, Low, Close, Volume) prices
  # Sample data
  df = yfin.Ticker(symbol+'.NS').history(period=prd, interval=intvl)[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  #df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
  #df = pd.DataFrame(data)
  #df.set_index('Date', inplace=True)

  # Calculate moving averages
  df['MA_short'] = df['Close'].rolling(window=sma).mean()
  df['MA_long'] = df['Close'].rolling(window=lma).mean()
  df['MA_xlong'] = df['Close'].rolling(window=xlma).mean()

  df = df[df.index >= start_date]
  col = tf + '_' + 'Flag'
  df[col] = 'R'
  df.loc[df['MA_short'] > df['MA_long'], col] = 'G'
  df = df[df.index > '31-Jan-2023']
  cols = [col]
  return df[df.index > '31-Jan-2023'][cols]

In [6]:

def get_stock_buy_sell(symbol):
  m_df = get_mtm_data(symbol, 'M')
  w_df = get_mtm_data(symbol, 'W')
  d_df = get_mtm_data(symbol, 'D')
  m_df['month'] = m_df.index.month
  w_df['week'] = w_df.index.week
  d_df['week'] = d_df.index.week
  d_df['month'] = d_df.index.month
  d_df = d_df.reset_index()
  cmb_df = pd.merge(d_df, w_df, on='week', how='outer')
  cmb_df = pd.merge(cmb_df, m_df, on='month', how='outer')
  cols = ['Date','D_Flag','W_Flag','M_Flag']
  cmb_df = cmb_df[cols]
  cmb_df.set_index('Date', inplace=True)
  df = yfin.Ticker(symbol+'.NS').history(period='1y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  df = df.join(cmb_df)
  df['F_Flag'] = 'R'
  df['G_Close'] = df['Close'].min()
  df.loc[(df['D_Flag'] == 'G') & (df['W_Flag'] == 'G') & (df['M_Flag'] == 'G'), 'F_Flag'] = 'G'
  df.loc[(df['F_Flag'] == 'G'), 'G_Close'] = df['Close'].max()
  df['Prev_F_Flag'] = df['F_Flag'].shift(1)
  df_reco = df[df['F_Flag'] != df['Prev_F_Flag']]
  df_reco['Signal'] = df_reco['F_Flag'].apply(lambda x: 'InSync' if x == 'G' else 'OutSync')
  df_reco.drop(['Open','Low','High','Volume'], axis=1, inplace=True)
  return df_reco.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id)
        df_tmp['Stock'] = stock_id
        df_buy_sell = df_buy_sell.append(df_tmp)
    df_buy_sell.drop(['D_Flag','W_Flag','M_Flag','G_Close','Prev_F_Flag'], axis=1, inplace=True)
    return df_buy_sell.sort_index()

df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,F_Flag,Signal,Stock
Date,,,,
2023-02-27 00:00:00+05:30,657.952759,R,OutSync,RAJESHEXPO
2023-03-01 00:00:00+05:30,462.269196,G,InSync,AUROPHARMA
2023-05-02 00:00:00+05:30,5427.100098,G,InSync,SANOFI
2023-05-02 00:00:00+05:30,1138.411133,G,InSync,INDUSINDBK
2023-05-02 00:00:00+05:30,2907.300537,G,InSync,DIXON
...,...,...,...,...
2024-01-18 00:00:00+05:30,487.549988,G,InSync,VAIBHAVGBL
2024-01-19 00:00:00+05:30,1073.550049,G,InSync,BLUESTARCO
2024-01-19 00:00:00+05:30,443.299988,G,InSync,INDIANB


In [7]:
mypf
#pd.merge(df_weekly_buy_sell, mypf, on="Symbol")

,Symbol,AvgCost,Shares,InPortfolio,Target,Breakout,MedianPE,NSECap,Criteria,Strategy,TFrame,LatestQtr,StarStock,BizFinTech,Conviction,Bucket,Category,Stock
0,3MINDIA,NaN,NaN,NaN,NaN,NaN,72.7,0.0,V40N,ATH,D,1,1,1.0,M,NaN,MISC,3MINDIA.NS
1,5PAISA,415.15,148.0,DM,810.0,NaN,65.0,0.0,V40N,BTT,W,1,1,1.0,M,NaN,FINANCE,5PAISA.NS
2,AARTIIND,487.04,167.0,SV,680.0,516.0,31.2,0.0,VR,NTT,W,1,0,0.0,L,NaN,CHEMICALS,AARTIIND.NS
3,AAVAS,1453.92,37.0,DM,1915.0,1666.0,57.7,0.0,VR,NTT,W,1,1,1.0,L,NaN,FINANCE,AAVAS.NS
4,AAVAS,1663.45,36.0,SV,1915.0,1666.0,57.7,0.0,VR,NTT,W,1,1,1.0,L,NaN,FINANCE,AAVAS.NS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,WIPRO,389.70,271.0,DM,533.0,NaN,20.9,1.0,VR,NTT,W,1,1,1.0,L,NaN,IT,WIPRO.NS
250,WIPRO,421.41,150.0,SV,533.0,NaN,20.9,1.0,VR,NTT,W,1,1,1.0,L,NaN,IT,WIPRO.NS
251,WSTCSTPAPR,NaN,NaN,NaN,NaN,NaN,NaN,0.0,VR,ATH,W,0,0,0.0,L,NaN,PAPER,WSTCSTPAPR.NS
252,YESBANK,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1,0,0.0,L,NaN,NaN,YESBANK.NS


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Signal'] == 'OutSync'].tail(20)

,Close,F_Flag,Signal,Stock
Date,,,,
2024-01-01 00:00:00+05:30,2575.350098,R,OutSync,LALPATHLAB
2024-01-01 00:00:00+05:30,1493.199951,R,OutSync,INDIGOPNTS
2024-01-01 00:00:00+05:30,648.700012,R,OutSync,HDFCLIFE
2024-01-01 00:00:00+05:30,2737.239990,R,OutSync,NESTLEIND
2024-01-01 00:00:00+05:30,285.049988,R,OutSync,ZEEL
2024-01-01 00:00:00+05:30,3478.649902,R,OutSync,PIIND
2024-01-01 00:00:00+05:30,1641.650024,R,OutSync,BATAINDIA
2024-01-01 00:00:00+05:30,5464.700195,R,OutSync,POLYCAB
2024-01-08 00:00:00+05:30,1395.650024,R,OutSync,ICICIGI


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Signal'] == 'InSync'].tail(20)

,Close,F_Flag,Signal,Stock
Date,,,,
2024-01-01 00:00:00+05:30,2593.949951,G,InSync,AKZOINDIA
2024-01-01 00:00:00+05:30,1362.199951,G,InSync,HAVELLS
2024-01-01 00:00:00+05:30,36.549999,G,InSync,TRIDENT
2024-01-01 00:00:00+05:30,903.650024,G,InSync,HAPPSTMNDS
2024-01-01 00:00:00+05:30,1051.599976,G,InSync,FINCABLES
2024-01-01 00:00:00+05:30,437.399994,G,InSync,AVANTIFEED
2024-01-01 00:00:00+05:30,38773.449219,G,InSync,PAGEIND
2024-01-01 00:00:00+05:30,799.250000,G,InSync,ROSSARI
2024-01-01 00:00:00+05:30,255.550003,G,InSync,BIOCON


In [10]:
df_weekly_buy_sell[df_weekly_buy_sell['Stock'] == 'PNB']

,Close,F_Flag,Signal,Stock
Date,,,,
2023-11-28 00:00:00+05:30,78.550003,G,InSync,PNB
